In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
# Visualize the behavior and function of individual CNN layers

# Function to create a model that outputs intermediate activations
def create_activation_model(model):
    layer_outputs = [layer.output for layer in model.layers]
    activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
    return activation_model

# Function to plot the feature maps
def plot_feature_maps(activations, layer_names, images_per_row=8):
    for layer_name, layer_activation in zip(layer_names, activations):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :, col * images_per_row + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size,
                             row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

# Create a model that outputs intermediate activations
activation_model = create_activation_model(created_models[0])

# Get the names of the layers
layer_names = [layer.name for layer in created_models[0].layers]

# Select an image from the test set
for example_spectrograms, example_spect_labels in test_spectrogram_ds.take(1):
    example_image = example_spectrograms[0]

# Get the activations of the selected image
activations = activation_model.predict(tf.expand_dims(example_image, axis=0))

# Plot the feature maps
plot_feature_maps(activations, layer_names)